In [13]:
################
##데이터 전처리##   
################

from PIL import Image
import os, glob

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/scissor"
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28) 
# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")


image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/rock"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("바위 이미지 resize 완료!")


image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/paper"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("보 이미지 resize 완료!")

이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/scissor
가위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/rock
바위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/paper
보 이미지 resize 완료!


In [2]:
###################
##훈련 데이터 적재##  루브릭 평가 문항 2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가? 10번 코드라인을 봐주세요!
##################


import numpy as np

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=3000   # 가위바위보 이미지 개수 각 1000장, 총 3000장.<<<<<<<<< 데이터셋의 다양성 확보. 총 10명의 손모양을 훈련셋으로 사용.
    img_size=28
    color=3 
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/"   

(x_train, y_train)=load_data(image_dir_path)    # 핵심 함수 : 훈련데이터의 입력과 레이블 생성하는 함수!!

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3000 입니다.
x_train shape: (3000, 28, 28, 3)
y_train shape: (3000,)


In [14]:
################
##데이터 정규화## Normalization! 루브릭 평가 문항 2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가? (V)
################

print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

최소값: 0  최대값: 255


In [15]:
#############
##모델 설계##
############

from tensorflow import keras

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
# Conv2D(16) 얼마나 다양한 이미지의 특징을 살펴볼 것인가? (입력 이미지가 다양할 수록 더 많은 특징 고려 필요)
# input_shape=(28,28,3) >> 입력 이미지의 형태 3 : 채널 정보 컬러사진 RGB니까 3
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
# Conv2D(32) 얼마나 다양한 이미지의 특징을 살펴볼 것인가? (입력 이미지가 다양할 수록 더 많은 특징 고려 필요)
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
# Dense(32) : 분류기 알고리즘을 얼마나 복잡하게 할 것인가? (복잡한 문제일 수록 이 숫자를 늘린다) 
model.add(keras.layers.Dense(3, activation='softmax'))
# 최종 분류기의 클래스 숫자 3 : 가위, 바위, 보

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [16]:
###############
##데이터 학습##
##############


model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
94/94 [==============================] - 0s 2ms/step - loss: 1.0322 - accuracy: 0.4520
Epoch 2/10
94/94 [==============================] - 0s 2ms/step - loss: 0.7512 - accuracy: 0.6723
Epoch 3/10
94/94 [==============================] - 0s 2ms/step - loss: 0.5464 - accuracy: 0.7883
Epoch 4/10
94/94 [==============================] - 0s 2ms/step - loss: 0.4047 - accuracy: 0.8580
Epoch 5/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2987 - accuracy: 0.8973
Epoch 6/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2119 - accuracy: 0.9353
Epoch 7/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1579 - accuracy: 0.9597
Epoch 8/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1313 - accuracy: 0.9623
Epoch 9/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1038 - accuracy: 0.9740
Epoch 10/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0722 - accuracy: 0.9850


In [18]:
#######################
## 테스트 데이터 적재##
#####################


import numpy as np
import os, glob

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/test/"

(x_test, y_test)=load_data(image_dir_path)

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


In [19]:
################
##데이터 정규화##
################
import numpy as np
print('최소값:',np.min(x_test), ' 최대값:',np.max(x_test))

x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

최소값: 0  최대값: 255


In [20]:
###########################
##훈련용 데이터와 결과 비교##  루브릭 평가문항 3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가? (X) 첫 시도는 40%대 나왔음. 
###########################

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 2.2888 - accuracy: 0.6033
test_loss: 2.288810968399048 
test_accuracy: 0.6033333539962769


In [31]:
######################
##하이퍼 파라미터 조정##   루브릭 평가문항 3. 분류모델의 test accuracy가 기준 (60%) 이상 높게 나왔는가? (V)  하이퍼 파라미터 조정결과
#######################                                                                                   
n_channel_1=64
n_channel_2=256
n_dense=512
n_train_epoch=15

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

# 모델 재훈련
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])


model.fit(x_train_norm, y_train, epochs=n_train_epoch)



Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 256)       147712    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 5, 5, 256)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               3277312   
_________________________________________________________________
dense_19 (Dense)             (None, 3)                

In [32]:
# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 1s - loss: 5.1771 - accuracy: 0.4933
test_loss: 5.17708683013916 
test_accuracy: 0.4933333396911621
